In [1]:
from torchtext.legacy.data import BucketIterator
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time

In [2]:
!nvidia-smi

Fri Jun 25 12:34:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.48       Driver Version: 451.48       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8     3W /  N/A |    701MiB /  4096MiB |     14%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [3]:
from dataset.cmu_qa_dataset import CmuQAndADataset
cmu_dataset = CmuQAndADataset(test_split=20)

In [4]:
print(f"Number of training examples: {len(cmu_dataset.train_dataset.examples)}")
print(f"Number of validation examples: {len(cmu_dataset.validation_dataset.examples)}")

Number of training examples: 2186
Number of validation examples: 547


In [5]:
vars(cmu_dataset.validation_dataset.examples[10])

{'source': ['when',
  'did',
  'the',
  'first',
  'educational',
  'works',
  'appear',
  'for',
  'the',
  'cello',
  '?'],
 'target': ['18th', 'century']}

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
BATCH_SIZE = 32

train_iterator, valid_iterator = BucketIterator.splits(
    (cmu_dataset.train_dataset, cmu_dataset.validation_dataset), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x: len(x.source),
    sort_within_batch=False)

In [8]:
from model.encoder import Encoder
from model.decoder import Decoder
from model.seq_to_seq import Seq2Seq
INPUT_DIM = len(cmu_dataset.Source.vocab)
OUTPUT_DIM = len(cmu_dataset.Target.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [9]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1872, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1293, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1293, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,829,965 trainable parameters


In [11]:
optimizer = optim.Adam(model.parameters())

In [12]:
TRG_PAD_IDX = cmu_dataset.Target.vocab.stoi[cmu_dataset.Target.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [13]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.source
        trg = batch.target
        
        optimizer.zero_grad()
        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.source
            trg = batch.target

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [15]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
cmu_dataset.train_dataset

torch.Size([8, 1293])
torch.Size([8])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([6, 1293])
torch.Size([6])
torch.Size([10, 1293])
torch.Size([10])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([13, 1293])
torch.Size([13])
torch.Size([9, 1293])
torch.Size([9])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([9, 1293])
torch.Size([9])
torch.Size([18, 1293])
torch.Size([18])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([14, 1293])
torch.Size([14])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([3, 1293])
torch.Size([3])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([3, 1293])
torch.Size([3])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([20, 1293])
torch.Size([20])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([2, 1293])
torch.Size([2])
torch.Size([3, 1293])
torch.Size([3])
to

KeyboardInterrupt: 

In [16]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    break
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'cmu-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [2]:
import os, pickle
with open('cmu-tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(cmu_dataset.Source.vocab, tokens)

NameError: name 'cmu_dataset' is not defined

In [1]:
import spacy
import numpy
nlp = spacy.load('en')
tokenizer_file = open('./cmu-tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)
vocab_string = list(nlp.vocab.strings)

model.load_state_dict(torch.load("./cmu-model.pt")).to("cpu")
model.eval()

def generate_answer(question):
    
    # tokenize the question 
    tokenized = [tok.text for tok in nlp.tokenizer(question)]
    print(tokenized)
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]
    # convert to tensor          
    tensor = torch.LongTensor(indexed).to("cpu")   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1)
    # Get the model prediction                  
    prediction = model(tensor, tensor).squeeze().argmax(dim=1)
    
    generated_sentence = ""
    for pred in prediction.numpy():
        generated_sentence += vocab_string[pred] + " "
    return generated_sentence

NameError: name 'pickle' is not defined

In [22]:
generate_answer("Was Abraham Lincoln the sixteenth President of the United States?")

['Was', 'Abraham', 'Lincoln', 'the', 'sixteenth', 'President', 'of', 'the', 'United', 'States', '?']


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR